In [1]:
import torch
from torch import nn
from torch import optim
import numpy as np
from tqdm.notebook import tqdm

In [2]:
a = np.load("npHot.npy")
b = np.load("npRatings.npy")
dataset = []
for i in range(len(a)):
    dataset.append((a[i],b[i]))

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(159, 318)
        self.fc2 = nn.Linear(318, 318)
        self.fc3 = nn.Linear(318,10)
        self.relu = nn.ReLU()
        self.tail = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return (self.tail(self.fc3(x)) * (torch.Tensor(list(range(1,11))).to(x.device))).sum(dim=1)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc11 = nn.Linear(3, 6)
        self.fc21 = nn.Linear(6, 3)
        self.fc12 = nn.Linear(28, 56)
        self.fc22 = nn.Linear(56, 10)
        self.fc13 = nn.Linear(93, 186)
        self.fc23 = nn.Linear(186, 10)
        self.fc14 = nn.Linear(35, 70)
        self.fc24 = nn.Linear(70, 10)
        self.fc2 = nn.Linear(33, 66)
        self.fc3 = nn.Linear(66,10)
        self.relu = nn.ReLU()
        self.tail = nn.Softmax(dim=1)
    def forward(self, x):
        f11 = self.relu(self.fc11(x[:,:3]))
        f12 = self.relu(self.fc12(x[:,3:31]))
        f13 = self.relu(self.fc13(x[:,31:124]))
        f14 = self.relu(self.fc14(x[:,124:159]))
        f21 = self.relu(self.fc21(f11))
        f22 = self.relu(self.fc22(f12))
        f23 = self.relu(self.fc23(f13))
        f24 = self.relu(self.fc24(f14))
        x = self.relu(self.fc2(torch.cat([f21,f22,f23,f24],dim=1)))
        return (self.tail(self.fc3(x)) * (torch.Tensor(list(range(1,11))).to(x.device))).sum(dim=1)

In [5]:
def hot(cat:list, size:int):
    a = [0 for _ in range(size)]
    for i in cat:
        if i != -1:
            a[int(i.item())] = 1
        else:
            break
    return a

In [6]:
def expand_data(in_data):
    a = []
    for item in in_data:
        lol = []
        lol.append([item[1]])
        lol.append([item[2]/2000])
        lol.append([item[4]/120])
        lol.append(hot(item[5:8], 28))
        lol.append(hot([item[8]], 93))
        lol.append(hot([item[9]], 35))
        lol = np.concatenate(lol)
        a.append(lol)
    return torch.Tensor(a)

In [8]:
loader = torch.utils.data.DataLoader(dataset, batch_size=1024, shuffle=True)
device = torch.device("cuda:0")
model = Net()
model = model.to(device)
optimizer = optim.SGD(model.parameters(), 0.1, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), 0.1)
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [40, 60, 80], 0.8)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [300, 600, 900], 0.1)
criteria = nn.MSELoss()
loss_min = 10000
iter_loader = tqdm(range(1000))
for _ in iter_loader:
    running_loss = 0.
    i = 0
    for data, labels in loader:
        # data = expand_data(data)
        data, labels = data.float().to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(data)
        # print(output.size())
        loss = criteria(output, labels.view(output.size()).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        i += 1
    # print(running_loss/i)
    scheduler.step()
    iter_loader.set_description(f"{running_loss/i:.4f}, {loss_min:.4f}")
    if running_loss/i < loss_min:
        loss_min = running_loss/i
    # print(output)
print(loss_min)

In [27]:
print(len(output))

501


In [18]:
def expand_data_lll(in_data):
    a = []
    for item in in_data:
        lol = []
        lol.append([item[1]])
        lol.append([item[2]/2000])
        lol.append([item[4]/120])
        lol.append(hot(item[5:8], 28))
        lol.append(hot([item[8]], 93))
        lol.append(hot([item[9]], 35))
        lol = np.concatenate(lol)
        a.append(lol)
    return a
bbb = np.load("npAttributes.npy")
ttt = np.array(expand_data_lll(bbb))
ttt.shape

(20981, 159)

In [20]:
np.save("npHot.npy", ttt)